# Image Classification with Pytorch

## Dataset and DataLoader

In [1]:
class Dataset(object):
    def __getitem__(self, index):
        pass

    def __len__(self):
        pass
    

In [1]:
import torchvision
from torchvision import transforms

train_data_path = './cat-dog-data/train/'
transforms = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transforms)


In [2]:
val_data_path = './cat-dog-data/val/'
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=transforms)

In [4]:
test_data_path = './cat-dog-data/test/'
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transforms)

In [5]:
from torch.utils import data 

batch_size = 1000
train_data_loader = data.DataLoader(
    train_data, 
    batch_size=batch_size,
    shuffle=True,
    num_workers=8,
    pin_memory=True,
    persistent_workers=True
)
val_data_loader = data.DataLoader(
    val_data, 
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
    persistent_workers=True
)
test_data_loader = data.DataLoader(test_data, batch_size=batch_size)

## SimpleNet model

In [6]:
import torch
torch.backends.cudnn.benchmark = True
import torch.nn as nn
import torch.nn.functional as F

class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2)

    def forward(self, x):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x) # as we will use CrossEntropyLoss() which calculates 'softmax' already, we don't use softmax here
        return x
simplenet = SimpleNet()

In [7]:
import torch.optim as optim
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)
loss = nn.CrossEntropyLoss()

## Actual training

In [8]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device='cpu'):
    print(device)
    model.to(device)
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0
        
        model.train()        
        for batch in train_loader:
            optimizer.zero_grad() # set weight-gradients to zero
            inputs, targets = batch # [batch,3,64,64], [batch]
            inputs = inputs.to(device, non_blocking=True)
            target = targets.to(device, non_blocking=True)
            
            output = model(inputs)
            
            loss = loss_fn(output, target)
            loss.backward() # gradients in weights
            optimizer.step() # update weights
            training_loss += loss.data.item() # loss: batch loss
        training_loss /= len(train_loader) # average batch loss per epoch

        model.eval()
        num_correct = 0
        num_examples = 0
        with torch.no_grad():
            for batch in val_loader:
                inputs, targets = batch
                inputs = inputs.to(device, non_blocking=True)
                output = model(inputs)
                targets = targets.to(device, non_blocking=True)
                loss = loss_fn(output, targets)
                valid_loss += loss.data.item()
                correct = torch.eq(torch.argmax(output, dim=1), targets)
                num_correct += torch.sum(correct).item()
                num_examples += correct.shape[0]
            valid_loss /= len(val_loader)
            print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}'
                  .format(epoch, training_loss, valid_loss, num_correct/num_examples))
        

In [9]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cuda


In [10]:
train(simplenet, optimizer, loss, train_data_loader, val_data_loader, epochs=100, device=device)

cuda
Epoch: 0, Training Loss: 0.75, Validation Loss: 0.84, accuracy = 0.40
Epoch: 1, Training Loss: 0.82, Validation Loss: 0.71, accuracy = 0.50
Epoch: 2, Training Loss: 0.84, Validation Loss: 0.90, accuracy = 0.60
Epoch: 3, Training Loss: 0.95, Validation Loss: 0.84, accuracy = 0.40
Epoch: 4, Training Loss: 0.60, Validation Loss: 1.08, accuracy = 0.70
Epoch: 5, Training Loss: 0.72, Validation Loss: 0.98, accuracy = 0.70
Epoch: 6, Training Loss: 0.61, Validation Loss: 0.73, accuracy = 0.80
Epoch: 7, Training Loss: 0.56, Validation Loss: 0.71, accuracy = 0.60
Epoch: 8, Training Loss: 1.69, Validation Loss: 0.59, accuracy = 0.60
Epoch: 9, Training Loss: 0.70, Validation Loss: 1.10, accuracy = 0.70
Epoch: 10, Training Loss: 1.39, Validation Loss: 1.49, accuracy = 0.50
Epoch: 11, Training Loss: 1.41, Validation Loss: 1.38, accuracy = 0.50
Epoch: 12, Training Loss: 0.97, Validation Loss: 1.12, accuracy = 0.60
Epoch: 13, Training Loss: 1.87, Validation Loss: 1.14, accuracy = 0.50
Epoch: 14, 

## Make prediction 

In [17]:
from PIL import Image
labels = ['cat', 'dog']
img = Image.open('./cat-dog-data/test/cat/16.jpg')
img = transforms(img)
img = img.unsqueeze(0) # [batch,3,64,64]
img = img.to(device)
simplenet.to(device)
prediction = simplenet(img)
prediction = prediction.argmax()
print(labels[prediction])

cat


## Saving both model and parameters

In [18]:
torch.save(simplenet, "tmp/simplenet")

RuntimeError: Parent directory tmp does not exist.

In [31]:
simplenet1 = torch.load("tmp/simplenet", weights_only=False)

## Saving only parameters

In [33]:
torch.save(simplenet.state_dict(), "tmp/simplenet2")

In [36]:
simplenet2 = SimpleNet()
simplenet2_state_dict = torch.load("tmp/simplenet2")
simplenet2.load_state_dict(simplenet2_state_dict)


<All keys matched successfully>

#### If model is modified, doesn't raise errors if there is one or two missing layers

In [38]:
simplenet2.load_state_dict(simplenet2_state_dict, strict=False)

<All keys matched successfully>